In [62]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np

# Загрузка данных
transactions = pd.read_csv("data/transactions_1.csv")
brands_categories = pd.read_csv("data/brands_categories.csv")

transactions['trxn_sum'] = transactions['trxn_sum'].str.replace(',','.').astype(np.float32)

# Преобразование даты
transactions['date'] = pd.to_datetime(transactions['date'])

# Агрегирование данных на уровне бренда по месяцам
brand_monthly_data = transactions.groupby(['brand_id', transactions['date'].dt.to_period('M')]).agg(
    trxn_sum=('trxn_sum', 'sum'),     # Общая сумма транзакций для бренда в месяц
    trxn_count=('trxn_count', 'sum')  # Общее количество покупок для бренда в месяц
).reset_index()

# Преобразуем столбец 'date' обратно в формат datetime для работы с временными признаками
brand_monthly_data['date'] = brand_monthly_data['date'].dt.to_timestamp()

brand_categories_df = pd.get_dummies(brands_categories.set_index('brand_id')['category_name']).groupby('brand_id').max().astype(np.int8)
brand_categories_df.columns = [f'cat_{x}' for x in brand_categories_df.columns]

brand_monthly_data = brand_monthly_data.merge(brand_categories_df, how='left', left_on='brand_id', right_index=True)

# brand_monthly_data = brand_monthly_data.merge(brands_categories, on='brand_id', how='left')
# brand_monthly_data['category_id'] = brand_monthly_data['category_id'].fillna(-1).astype(np.int8)


# Создание новых признаков
# 1. Средняя и медианная сумма транзакций за предыдущие 3 месяца
brand_monthly_data['mean_trxn_sum_3m'] = brand_monthly_data.groupby('brand_id')['trxn_sum'].transform(
    lambda x: x.rolling(window=3, min_periods=1).mean().shift(1)
)
brand_monthly_data['median_trxn_sum_3m'] = brand_monthly_data.groupby('brand_id')['trxn_sum'].transform(
    lambda x: x.rolling(window=3, min_periods=1).median().shift(1)
)

# 2. Скользящее среднее количества транзакций за последние 3 месяца
brand_monthly_data['mean_trxn_count_3m'] = brand_monthly_data.groupby('brand_id')['trxn_count'].transform(
    lambda x: x.rolling(window=3, min_periods=1).mean().shift(1)
)

# 3. Сезонный признак (месяц), который поможет учесть сезонные колебания в расходах
brand_monthly_data['month'] = brand_monthly_data['date'].dt.month

# 4. Признаки из предыдущего месяца
brand_monthly_data['prev_month_trxn_sum'] = brand_monthly_data.groupby('brand_id')['trxn_sum'].shift(1)
brand_monthly_data['prev_month_trxn_count'] = brand_monthly_data.groupby('brand_id')['trxn_count'].shift(1)




# Заполнение пропусков
brand_monthly_data.fillna(0, inplace=True)

# Определение признаков и целевых переменных
features = [
    'brand_id', 'prev_month_trxn_sum', 'prev_month_trxn_count', 'mean_trxn_sum_3m', 
    'median_trxn_sum_3m', 'mean_trxn_count_3m', 'month' #  , 'category_id'
] + [x for x in brand_monthly_data.columns if 'cat_' in x]
target_trxn_sum = 'trxn_sum'
target_trxn_count = 'trxn_count'

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train_trxn_sum, y_test_trxn_sum = train_test_split(
    brand_monthly_data[features], brand_monthly_data[target_trxn_sum], test_size=0.2, random_state=42
)
_, _, y_train_trxn_count, y_test_trxn_count = train_test_split(
    brand_monthly_data[features], brand_monthly_data[target_trxn_count], test_size=0.2, random_state=42
)

In [65]:
# Обучение модели для прогнозирования суммы транзакций
model_trxn_sum = CatBoostRegressor(iterations=3000, depth=6, loss_function="MAE", eval_metric="MAE", learning_rate=0.03, cat_features=['brand_id'], verbose=100, random_state=42)
model_trxn_sum.fit(X_train, y_train_trxn_sum, eval_set=(X_test, y_test_trxn_sum))

# Обучение модели для прогнозирования количества покупок
model_trxn_count = CatBoostRegressor(iterations=3000, depth=6, loss_function="MAE", eval_metric="MAE", learning_rate=0.03, cat_features=['brand_id'], verbose=100, random_state=42)
model_trxn_count.fit(X_train, y_train_trxn_count, eval_set=(X_test, y_test_trxn_count))

# Оценка моделей на тестовой выборке
y_pred_trxn_sum = model_trxn_sum.predict(X_test)
y_pred_trxn_count = model_trxn_count.predict(X_test)

print("MAE для прогноза суммы транзакций:", mean_absolute_error(y_test_trxn_sum, y_pred_trxn_sum))
print("MAE для прогноза количества покупок:", mean_absolute_error(y_test_trxn_count, y_pred_trxn_count))

0:	learn: 1994656.6260741	test: 2134464.8939752	best: 2134464.8939752 (0)	total: 5.07ms	remaining: 15.2s
100:	learn: 784717.1702001	test: 886298.0410181	best: 886298.0410181 (100)	total: 438ms	remaining: 12.6s
200:	learn: 465823.1213951	test: 536467.1737778	best: 536467.1737778 (200)	total: 828ms	remaining: 11.5s
300:	learn: 422295.0911266	test: 501464.7494911	best: 501464.7494911 (300)	total: 1.22s	remaining: 10.9s
400:	learn: 387200.9923136	test: 471188.2210795	best: 471188.2210795 (400)	total: 1.62s	remaining: 10.5s
500:	learn: 360606.3871246	test: 447743.7637974	best: 447743.7637974 (500)	total: 2.01s	remaining: 10s
600:	learn: 341359.4333187	test: 432695.3339515	best: 432584.5746282 (598)	total: 2.41s	remaining: 9.61s
700:	learn: 320748.6333465	test: 420626.3769537	best: 420282.0818524 (699)	total: 2.81s	remaining: 9.21s
800:	learn: 303219.5329271	test: 410204.1749405	best: 410162.5340784 (798)	total: 3.2s	remaining: 8.78s
900:	learn: 289482.2695832	test: 405564.8308131	best: 4053